# 기본옵션

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pip install KoNLPy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 2.9MB 36.0MB/s 
     |████████████████████████████████| 92kB 7.1MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [0]:
import pandas as pd 
import numpy as np
import re
import os
import itertools
import json
from collections import OrderedDict

import seaborn as sns 
import matplotlib as mpl
import matplotlib.pyplot as plt 

import warnings
warnings.filterwarnings("ignore")

# 한국어 전처리
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import json
import sys

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras_preprocessing import sequence

from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

from sklearn.metrics import accuracy_score

# 모델링
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from sklearn.naive_bayes import MultinomialNB
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv1D

from keras import metrics
from keras import optimizers
from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense

import gensim


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


# Text Feature Extraction

# Kkma

## 전처리

### 텍스트 정규화 (노이즈 제거)

In [0]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/small_total_train_set.csv", encoding='utf-8')

In [0]:
# 불용어사전 읽어오기
f = open("/content/drive/My Drive/Colab Notebooks/ko/stopwords.txt", 'r', encoding='utf-8')
stopwords = []
for line in f.readlines():
    stopwords.append(line.rstrip())
f.close()

# #분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')

clean_name_lst=[]
for i in range(len(df)):
    a = re.sub(han,'',df['video_name'].iloc[i])
    clean_name_lst.append(a)
clean_name_lst=[]
for i in range(len(df)):
    a = re.sub(emoji_pattern,'',df['video_name'].iloc[i])
    b = re.sub(han,'',a)
    
    clean_name_lst.append(b)
    
df['clean_name']=clean_name_lst


In [0]:
from konlpy.tag import Kkma
kkma = Kkma()

clean_word = [] 
for i in df['clean_name']:
    text = re.sub("[12?~3.,()''->\n]", '', i)
    word_text = kkma.nouns(text)
    if True: word_text = [token for token in word_text if not token in stopwords]
        
    clean_word.append(' '.join(word_text))
    
df['clean_name'] = clean_word
df['kkma_tok'] = df['clean_name'].apply(lambda x : kkma.nouns(x))
df.head()



KeyError: ignored

In [0]:
kkma_data = df[['category_id','video_name', 'channel_name', 'kkma_tok']]

### 토큰화 및 단어집합

In [0]:
# 토크나이저 사용
# keras의 tokenizer는 default값이 띄어쓰기 구분이므로 kkma로 토큰화된 단어들만 뽑아오는게 맞음
kkma_vocab = Tokenizer()
kkma_vocab.fit_on_texts(kkma_data['kkma_tok'])

In [0]:
print(len(kkma_vocab.word_index))
print(kkma_vocab.word_index)

21185
{'한국': 1, '경제': 2, '레': 3, '고양이': 4, '개의': 5, '개의레': 6, '김': 7, '문': 8, '법': 9, '대': 10, '강': 11, '재인': 12, '미국': 13, '정권': 14, '전': 15, '이슈': 16, '게임': 17, '대표': 18, '이유': 19, '증시': 20, '만들기': 21, '주식': 22, '쉪': 23, '문재인': 24, '중국': 25, '남자': 26, '정치': 27, '황': 28, '방법': 29, '안': 30, '미': 31, '축구': 32, '조국': 33, '교': 34, '트럼프': 35, '국민': 36, '코로나': 37, '북한': 38, '합': 39, '맛': 40, '자유': 41, '영상': 42, '세계': 43, '야구': 44, '투자': 45, '강아지': 46, '냥': 47, '정은': 48, '민': 49, '전략': 50, '유': 51, '분석': 52, '위': 53, '바': 54, '선수': 55, '대한': 56, '팁': 57, '대한민국': 58, '민국': 59, '공개': 60, '반응': 61, '기자': 62, '핵': 63, '글로벌': 64, '박': 65, '운동': 66, '개': 67, '요리': 68, '손': 69, '총선': 70, '부동산': 71, '볶음': 72, '코': 73, '아기': 74, '정리': 75, '동산': 76, '부부': 77, '정부': 78, '주의': 79, '국가': 80, '중': 81, '부부동산': 82, '대통령': 83, '황금': 84, '차': 85, '최고': 86, '프로': 87, '무엇': 88, '시장': 89, '길': 90, '뭐': 91, '국': 92, '우': 93, '북': 94, '해외': 95, '피': 96, '편': 97, '영화': 98, '당': 99, '무침': 100, '정': 101, '이것': 102, '

In [0]:
threshold = 3
total_cnt = len(kkma_vocab.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in kkma_vocab.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 21185
등장 빈도가 2번 이하인 희귀 단어의 수: 14953
단어 집합에서 희귀 단어의 비율: 70.5829596412556
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 15.030188551961412


In [0]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
kkma_vocab_size = total_cnt - rare_cnt 
print('단어 집합의 크기 :', kkma_vocab_size)

단어 집합의 크기 : 6232


In [0]:
# 빈도수가 3미만인 단어를 w라고 저장
kkma_freq = [w for w,c in kkma_vocab.word_counts.items() if c < 3]

for w in kkma_freq:
    del kkma_vocab.word_index[w]
    del kkma_vocab.word_counts[w]

kkma_vocab_list = list(kkma_vocab.word_index.keys())

In [0]:
kkma_all = kkma_data['kkma_tok'].tolist()

titles = list(kkma_data.video_name)
id_num = list(kkma_data.category_id)

In [0]:
len(id_num)

13685

In [0]:
len(kkma_all)

13685

### 리스트 빈값 지우기

In [0]:

nan_lst =[]
for i in range(len(kkma_all)):
  if not kkma_all[i]:
    nan_lst.append(i)

nnnn = np.arange(len(kkma_all)) # 이게 kkma_all의 인덱스
tttt = set(nnnn).difference(set(nan_lst)) # 인덱스에서 nan_lst뺀거!

kkma_all = [kkma_all[i] for i in tttt]
id_num = [id_num[i] for i in tttt]
titles = [titles[i] for i in tttt]

In [0]:
# kkma_vocab_size : kkma 단어집합의 크기 6232
# kkma_vocab_list : kkma 사용할 단어 집합 리스트 형태
# kkma_freq : kkma 사용하지 않는 단어 리스트
# kkma_data의 컬럼명 : category_id, channel_name, thumbnail, kkma_tok

## 단어 임베딩

### word2vec

In [0]:
# sg=0 (cbow), sg=1 (skip-gram)
from gensim.models import Word2Vec
kkma_w2v = Word2Vec(sentences = kkma_all, size = 100, window = 2, min_count = 1, workers = 4,sample=1e-3, iter=5, sg = 1)

In [0]:
def get_features(words, model, num_features):
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features), dtype=np.float32)
    
    num_words = 0
    # 어휘 사전 준비
    index2word_set = set(model.wv.index2word)
    
    for w in words:
        if w in index2word_set:
            num_words = 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model[w])
            
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [0]:
def get_dataset(data, model, num_features):
    dataset = list()
    
    for s in data:
        dataset.append(get_features(s, model, num_features))
        
    reviewFeatureVecs = np.stack(dataset)
    return reviewFeatureVecs

In [0]:
kkma_w2v = get_dataset(kkma_all, kkma_w2v, 100)
kkma_w2v = kkma_w2v.astype('float32')

### Fasttext

In [0]:
from gensim.models import FastText
fxt = FastText(sentences = kkma_all, size = 100, window = 4, min_count = 1, workers = 4,sample=1e-3, iter=5, sg = 1)

kkma_fxt = get_dataset(kkma_all, fxt, 100)
kkma_fxt = kkma_fxt.astype('float32')

In [0]:
print(kkma_w2v.shape)
print(kkma_fxt.shape)

(13519, 100)
(13519, 100)


In [0]:
np.save("/content/drive/My Drive/딥러닝프로젝트_유튜브/데이터/단어임베딩/kkma_w2v", kkma_w2v) # save.npy
np.save("/content/drive/My Drive/딥러닝프로젝트_유튜브/데이터/단어임베딩/kkma_fxt", kkma_fxt) # save.npy

## 1차 모델 평가

## word2vec

### Logistic Regression

In [0]:
from sklearn.model_selection import train_test_split

X = kkma_w2v
y = np.array(id_num)

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, 
                                                    random_state=RANDOM_SEED)

In [0]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

predicted = lgs.predict(X_test)

print("kkma word2vec Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy

kkma word2vec Accuracy: 0.779586


### SVC

In [0]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear', C=1.0)
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
accuracy_score(y_test,y_pred)

print("kkma word2vec SVC Accuracy: %f" % accuracy_score(y_test,y_pred))

kkma word2vec SVC Accuracy: 0.812130


## Fasttext

### Logistic Regression

In [0]:
from sklearn.model_selection import train_test_split

X = kkma_fxt
y = np.array(id_num)

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, 
                                                    random_state=RANDOM_SEED)

In [0]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

predicted = lgs.predict(X_test)

print("kkma fasttext LR Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy

### SVC

In [0]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear', C=1.0)
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
accuracy_score(y_test,y_pred)

print("kkma fasttext SVC Accuracy: %f" % accuracy_score(y_test,y_pred))

# Okt

## 전처리

### 텍스트 정규화 (노이즈 제거)

In [0]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/small_total_train_set.csv", encoding='utf-8')

In [0]:
# 불용어사전 읽어오기
f = open("/content/drive/My Drive/Colab Notebooks/ko/stopwords.txt", 'r', encoding='utf-8')
stopwords = []
for line in f.readlines():
    stopwords.append(line.rstrip())
f.close()

# #분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')

clean_name_lst=[]
for i in range(len(df)):
    a = re.sub(han,'',df['video_name'].iloc[i])
    clean_name_lst.append(a)
clean_name_lst=[]
for i in range(len(df)):
    a = re.sub(emoji_pattern,'',df['video_name'].iloc[i])
    b = re.sub(han,'',a)
    
    clean_name_lst.append(b)
    
df['clean_name']=clean_name_lst


In [0]:
from konlpy.tag import Okt
okt = Okt()

clean_word = [] 
for i in df['clean_name']:
    text = re.sub("[12?~3.,()''->\n]", '', i)
    word_text = okt.nouns(text)
    if True: word_text = [token for token in word_text if not token in stopwords]
        
    clean_word.append(' '.join(word_text))
    
df['clean_name'] = clean_word
df['okt_tok'] = df['clean_name'].apply(lambda x : okt.nouns(x))

okt_data = df[['category_id','video_name', 'channel_name', 'okt_tok']]

### 토큰화 및 단어집합

In [0]:
# 토크나이저 사용
# keras의 tokenizer는 default값이 띄어쓰기 구분이므로 okt로 토큰화된 단어들만 뽑아오는게 맞음
okt_vocab = Tokenizer()
okt_vocab.fit_on_texts(okt_data['okt_tok'])

In [0]:
threshold = 3
total_cnt = len(okt_vocab.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in okt_vocab.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 13024
등장 빈도가 2번 이하인 희귀 단어의 수: 7888
단어 집합에서 희귀 단어의 비율: 60.56511056511057
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 11.032626606613205


In [0]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
okt_vocab_size = total_cnt - rare_cnt 
print('단어 집합의 크기 :', okt_vocab_size)

단어 집합의 크기 : 5136


In [0]:
# 빈도수가 3미만인 단어를 w라고 저장
okt_freq = [w for w,c in okt_vocab.word_counts.items() if c < 3]

for w in okt_freq:
    del okt_vocab.word_index[w]
    del okt_vocab.word_counts[w]

okt_vocab_list = list(okt_vocab.word_index.keys())

In [0]:
okt_all = okt_data['okt_tok'].tolist()

titles = list(okt_data.video_name)
id_num = list(okt_data.category_id)

### 리스트 빈값 지우기

In [0]:

nan_lst =[]
for i in range(len(okt_all)):
  if not okt_all[i]:
    nan_lst.append(i)

nnnn = np.arange(len(okt_all)) # 이게 kkma_all의 인덱스
tttt = set(nnnn).difference(set(nan_lst)) # 인덱스에서 nan_lst뺀거!

okt_all = [okt_all[i] for i in tttt]
id_num = [id_num[i] for i in tttt]
titles = [titles[i] for i in tttt]

## 단어 임베딩

In [0]:
# sg=0 (cbow), sg=1 (skip-gram)
from gensim.models import Word2Vec
w2v = Word2Vec(sentences = okt_all, size = 100, window = 2, min_count = 1, workers = 4,sample=1e-3, iter=5, sg = 1)

In [0]:
from gensim.models import FastText
fxt = FastText(sentences = okt_all, size = 100, window = 4, min_count = 1, workers = 4,sample=1e-3, iter=5, sg = 1)

## 문장 특성 추출 (단어 벡터의 평균값)

In [0]:
def get_features(words, model, num_features):
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features), dtype=np.float32)
    
    num_words = 0
    # 어휘 사전 준비
    index2word_set = set(model.wv.index2word)
    
    for w in words:
        if w in index2word_set:
            num_words = 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model[w])
            
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [0]:
def get_dataset(data, model, num_features):
    dataset = list()
    
    for s in data:
        dataset.append(get_features(s, model, num_features))
        
    reviewFeatureVecs = np.stack(dataset)
    return reviewFeatureVecs

In [0]:
okt_w2v = get_dataset(okt_all, w2v, 100)

In [0]:
okt_fxt = get_dataset(okt_all, fxt, 100)

In [0]:
np.save("/content/drive/My Drive/딥러닝프로젝트_유튜브/데이터/단어임베딩/okt_w2v", okt_w2v) # save.npy
np.save("/content/drive/My Drive/딥러닝프로젝트_유튜브/데이터/단어임베딩/okt_fxt", okt_fxt) # save.npy

## 1차 모델 평가

## word2vec

### Logistic Regression

In [0]:
from sklearn.model_selection import train_test_split

X = okt_w2v
y = np.array(id_num)

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, 
                                                    random_state=RANDOM_SEED)

In [0]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

predicted = lgs.predict(X_test)

print("okt word2vec Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy

okt word2vec Accuracy: 0.751836


### SVC

In [0]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear', C=1.0)
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
accuracy_score(y_test,y_pred)

print("okt word2vec SVC Accuracy: %f" % accuracy_score(y_test,y_pred))

okt word2vec SVC Accuracy: 0.790749


## Fasttext

### Logistic Regression

In [0]:
from sklearn.model_selection import train_test_split

X = okt_fxt
y = np.array(id_num)

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, 
                                                    random_state=RANDOM_SEED)

In [0]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

predicted = lgs.predict(X_test)

print("okt fasttext LR Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy

okt fasttext LR Accuracy: 0.824156


### SVC

In [0]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear', C=1.0)
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
accuracy_score(y_test,y_pred)

print("okt fasttext SVC Accuracy: %f" % accuracy_score(y_test,y_pred))

okt fasttext SVC Accuracy: 0.849119
